In [1]:
import os
from IPython.display import display, Markdown, clear_output
from dotenv import load_dotenv
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.docstore.document import Document
from langchain.agents import Tool

from settings import vectorizing_params, retriever_params


# Chargement des clés d'API se trouvant dans le fichier .env.  
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

model = ChatOllama(model="llama3.2", temperature=0)
#model = ChatDeepSeek(model="deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))

# Modèle spécialisé pour convertir du texte en vecteurs (https://ollama.com/library/nomic-embed-text).
# Il existe d'autres modèles d'embeddings (comme "all-MiniLM-L6-v2", "text-embedding-ada-002", etc.) 
# avec des performances et dimensions variées selon les cas d’usage (recherche sémantique, classification, etc.).
embedder = OllamaEmbeddings(model="nomic-embed-text")

### Creation du vectorstore et du retriever

In [ ]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "data")
db_dir = os.path.join(current_dir, "db")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=vectorizing_params['chunk_size'],
    chunk_overlap=vectorizing_params['chunk_overlap']
)

documents = []

if not os.path.exists(db_dir):
    print("Initializing vector store...")   


    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                with open(file_path, "r", encoding="utf-8") as f:
                    full_text = f.read()

                # Extract themes and subthemes from the file structure
                relative_path = os.path.relpath(file_path, "data")
                parts = relative_path.split(os.sep)
                large_theme = parts[0]
                theme = parts[1]
                subtheme = parts[2].replace(".txt", "")

                # Chunk splitting
                chunks = text_splitter.split_text(full_text)
                for i, chunk in enumerate(chunks):
                    documents.append(
                        Document(
                            page_content=chunk,
                            metadata={
                                "large_theme": large_theme,
                                "theme": theme,
                                "subtheme": subtheme,
                                "chunk_id": i,
                                "source": file_path
                            }
                        )
                    )

vectorstore = Chroma.from_documents(
    documents,
    embedding=embedder,
    collection_name="droits",
    persist_directory=db_dir
)

#db = Chroma.from_documents(chunks, embedder, persist_directory=db_dir)

print(f"Vectorstore created with {len(documents)} chunks.")

#db = Chroma(persist_directory=db_dir, embedding_function=embedder)

retriever = vectorstore.as_retriever(
    search_type=retriever_params['search_type'],
    search_kwargs=retriever_params['search_kwargs']
)

Initializing vector store...


In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(llm=model, retriever=retriever)

chat_history = [
    SystemMessage(content="Tu es un assistant qui aide à trouver des informations concernant les droits disponibles.")
]

def ask_rag(query: str) -> str:

    relevant_chunks = retriever.invoke(query)
            
    input_message = (
        "Voici des documents qui vont t'aider à répondre à la question : "
        + query
        + "\n\nDocuments pertinents : \n"
        + "\n\n".join([chunk.page_content for chunk in relevant_chunks])
        + "\n\nDonne une réponse basée uniquement sur les documents qui te sont fournis."
    )

    result = qa_chain({
        "question": query,
        "chat_history": chat_history
    })
    chat_history.append((query, result["answer"]))
    return result["answer"]

rag_tool = Tool(
    name="consult_droit",
    func=ask_rag,
    description="Répond à des questions sur les droits sociaux (APL, RSA, etc.). Fournit des réponses fiables extraites de documents organisés par thème."
)